# Overview
Steps to build the network:

1. Load the training data and do a train/validation split.
2. Preprocess data.
3. Build a convolutional neural network to classify traffic signs.
4. Build a feedforward neural network to classify traffic signs.
5. Evaluate performance of final neural network on testing data.

Keep an eye on the network’s accuracy over time. Once the accuracy reaches the 98% range, you can be confident that you’ve built and trained an effective model.

In [1]:
import pandas as pd
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
import math

from keras.models import Sequential
from keras.layers import Dense, Activation

import json

Using TensorFlow backend.


Load the Data

In [2]:
# Load the data by reading the logfile

data = pd.read_csv('data/data/driving_log.csv',dtype={'center': str, 'left':str,'right':str,'steering': np.float32,'throttle': np.float32,'brake': np.float32,'speed': np.float32})
                   #converters={'category',"Price":int} dtype='category')
print(data.dtypes)

X_train = data['center']
y_train = data['steering']

print('Training data size = ', len(X_train))
print('Training labels size = ',len(y_train))
print('Data loaded')

center       object
left         object
right        object
steering    float32
throttle    float32
brake       float32
speed       float32
dtype: object
Training data size =  8036
Training labels size =  8036
Data loaded


In [19]:
# preprocess all data and save to pickle files
from PIL import Image
import matplotlib.pyplot as plt
import pickle
    
def normalise(im):
    width,height = im.size
    im = np.array(im.resize((round(width*0.5),round(height*0.5)), Image.ANTIALIAS)) # reduce to half size
    im = im/255 - 0.5 # normalise data
    return im
        
def append_pickle (pickle_filename, data):
    with open(pickle_filename,'ab') as wfp:
        pickle.dump(data, wfp)

def process_images (data, pickle_filename):
    X_train_im = []
    for filename in data:
        #print('data/data/'+filename)
        im = Image.open('data/data/'+filename)
        #plt.imshow(im);
        #plt.show()
        im = normalise(im)
        X_train_im.append(im)
    return X_train_im
        #append_pickle(pickle_filename, im)
    #print('Data saved to pickle file:',pickle_filename)
    
def process_labels (data, pickle_filename):
    append_pickle(pickle_filename, data)
    print('Data saved to pickle file:',pickle_filename)

X_train_im = process_images (X_train,'data.p') # note: here images are saved one by one
#process_labels (y_train[1:6],'labels.p') # note: labels are saved all in one go
print ('Images processed and stored in array')

Images processed and stored in array


In [20]:
#save data to pickle files
pickle_filename = 'data.p'
with open(pickle_filename,'wb') as wfp:
    pickle.dump(X_train_im, wfp)
print('data saved to pickle file')
pickle_filename = 'labels.p'
with open(pickle_filename,'wb') as wfp:
    pickle.dump(y_train, wfp)
print('labels saved to pickle file')

data saved to pickle file
labels saved to pickle file


In [23]:
# read data from pickle files
X_train = []
with open('data.p','rb') as rfp:
    X_train = pickle.load(rfp)
y_train = []
with open('labels.p','rb') as rfp:
    y_train = pickle.load(rfp)
print(len(X_train))
print(len(y_train))

8036
8036


In [24]:
from sklearn.model_selection import train_test_split

# split data into train, validate and test data
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
print('Data splitted into train and validation data')
print('X_train contains just the path at the moment')
print('X_val contains just the path at the moment')

Data splitted into train and validation data
X_train contains just the path at the moment
X_val contains just the path at the moment


In [25]:
print('Training data size = ', len(X_train))
print('Training labels size = ',len(y_train))
print('Validation data size = ',len(X_val))
print('Validation labels size = ',len(X_val))

Training data size =  6428
Training labels size =  6428
Validation data size =  1608
Validation labels size =  1608


In [18]:
from random import shuffle
batch_size = 10
x = [i for i in range(len(X_val))]
shuffle(x)
indices = x[0:batch_size]
    
print(indices)

[365, 703, 967, 12, 78, 1029, 1552, 421, 474, 1172]


(80, 160, 3)
38400


In [40]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
# define the model
print('Input shape:',(X_train[0].shape))
model = Sequential()
model.add(Conv2D(32, 3, 3, input_shape=X_train[0].shape))
model.add(MaxPooling2D((2,2)))
model.add((Dropout(0.5)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(43, activation='softmax'))
# for a mean squared error regression problem
model.compile(optimizer='rmsprop',
              loss='mse')
print('Model defined')

Input shape: (80, 160, 3)
Model defined


In [39]:
#train the model
print('')


In [ ]:
# convert model to json format
json_string = model.to_json()
model = model_from_json(json_string)

#save model to files
model.save_weights(model.h5)
with open('model.json', 'w') as outfile:
    json.dump(model, outfile)